In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [3]:
# Cargar el conjunto de datos
data = pd.read_csv('heart_2020_cleaned.csv')

# Convertir la variable objetivo a binaria (1 = Yes, 0 = No)
data['HeartDisease'] = data['HeartDisease'].apply(lambda x: 1 if x == 'Yes' else 0)

In [4]:
# Convertir variables categóricas a numéricas con Label Encoding
categorical_columns = data.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [5]:
# Separar características (X) y variable objetivo (y)
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']

# Dividir los datos en entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

# Aplicar el reequilibrio de clases utilizando class_weight='balanced'
model_balanced = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

# Definir los hiperparámetros a ajustar (C para regularización)
param_grid = {'C': [0.1, 1, 10, 100, 1000]}

# GridSearchCV para encontrar el mejor valor de C
grid_search = GridSearchCV(model_balanced, param_grid, cv=5, scoring='f1', verbose=0)
grid_search.fit(X_train, y_train)

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Evaluar en el conjunto de validación
y_val_pred_best = best_model.predict(X_val)
accuracy_best = accuracy_score(y_val, y_val_pred_best)
classification_report_best = classification_report(y_val, y_val_pred_best)

print(f"Mejor parámetro C: {grid_search.best_params_}")
print(f"Accuracy en validación: {accuracy_best}")
print(f"Reporte de clasificación:\n{classification_report_best}")


Mejor parámetro C: {'C': 100}
Accuracy en validación: 0.7387062477850278
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.97      0.74      0.84     43863
           1       0.21      0.77      0.33      4106

    accuracy                           0.74     47969
   macro avg       0.59      0.75      0.59     47969
weighted avg       0.91      0.74      0.79     47969



In [7]:
from sklearn.model_selection import GridSearchCV, cross_val_score


# Aplicar el reequilibrio de clases utilizando class_weight='balanced'
model_balanced = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

# Definir los hiperparámetros a ajustar (C para regularización)
param_grid = {'C': [0.1, 1, 10, 100, 1000]}

# GridSearchCV para encontrar el mejor valor de C con validación cruzada
grid_search = GridSearchCV(model_balanced, param_grid, cv=5, scoring='f1', verbose=0)
grid_search.fit(X_train, y_train)

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Realizar validación cruzada con el mejor modelo
cross_val_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='f1')

# Evaluar el mejor modelo en el conjunto de validación
y_val_pred_best = best_model.predict(X_val)
accuracy_best = accuracy_score(y_val, y_val_pred_best)
classification_report_best = classification_report(y_val, y_val_pred_best)

# Resultados
print(f"Mejor parámetro C: {grid_search.best_params_}")
print(f"Accuracy en validación: {accuracy_best}")
print(f"Reporte de clasificación:\n{classification_report_best}")
print(f"Puntuaciones de validación cruzada (F1-score): {cross_val_scores}")
print(f"Promedio de F1-score en validación cruzada: {cross_val_scores.mean()}")


Mejor parámetro C: {'C': 100}
Accuracy en validación: 0.7387062477850278
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.97      0.74      0.84     43863
           1       0.21      0.77      0.33      4106

    accuracy                           0.74     47969
   macro avg       0.59      0.75      0.59     47969
weighted avg       0.91      0.74      0.79     47969

Puntuaciones de validación cruzada (F1-score): [0.33698662 0.33765935 0.33316242 0.33862799 0.335942  ]
Promedio de F1-score en validación cruzada: 0.33647567652478105
